In [1]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role
import boto3
bucket = 'sagemaker-pursu'
region = 'us-east-2'

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

In [2]:
# Train the relevance model using by passing the path to the S3 bucket with the training data
layer3 = SKLearn(entry_point='train_script_layer3.py',train_instance_type='ml.m4.xlarge',role=role,source_dir='/home/ec2-user/SageMaker/pursu/model',output_path='s3://sagemaker-pursu-model-data/model-artifacts')
layer3.fit({'train': 's3://sagemaker-pursu'})

This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-06-27 00:20:35 Starting - Starting the training job...
2020-06-27 00:20:37 Starting - Launching requested ML instances......
2020-06-27 00:21:43 Starting - Preparing the instances for training...
2020-06-27 00:22:35 Downloading - Downloading input data......
2020-06-27 00:23:29 Training - Training image download completed. Training in progress..2020-06-27 00:23:30,323 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-06-27 00:23:30,326 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-27 00:23:30,337 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-06-27 00:23:30,604 sagemaker-containers INFO     Module train_script_layer3 does not provide a setup.py. 
Generating setup.py
2020-06-27 00:23:30,605 sagemaker-containers INFO     Generating setup.cfg
2020-06-27 00:23:30,605 sagemaker-containers INFO     Generating MANIFEST.in
2020-06-27 00:23:30,605 sagemaker-containers INFO    


2020-06-27 00:24:05 Uploading - Uploading generated training model2020-06-27 00:24:03,986 sagemaker-containers INFO     Reporting training SUCCESS

2020-06-27 00:24:12 Completed - Training job completed
Training seconds: 97
Billable seconds: 97


In [6]:
# Train the categorization model using by passing the path to the S3 bucket with the training data
layer4 = SKLearn(entry_point='train_script_layer4.py',train_instance_type='ml.m4.xlarge',role=role,source_dir='/home/ec2-user/SageMaker/pursu/model',output_path='s3://sagemaker-pursu-model-data/model-artifacts')
layer4.fit({'train': 's3://sagemaker-pursu'})

This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-06-27 00:34:53 Starting - Starting the training job...
2020-06-27 00:34:55 Starting - Launching requested ML instances......
2020-06-27 00:35:58 Starting - Preparing the instances for training...
2020-06-27 00:36:47 Downloading - Downloading input data...
2020-06-27 00:37:23 Training - Training image download completed. Training in progress..2020-06-27 00:37:23,297 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-06-27 00:37:23,299 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-06-27 00:37:23,311 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-06-27 00:37:23,580 sagemaker-containers INFO     Module train_script_layer4 does not provide a setup.py. 
Generating setup.py
2020-06-27 00:37:23,580 sagemaker-containers INFO     Generating setup.cfg
2020-06-27 00:37:23,580 sagemaker-containers INFO     Generating MANIFEST.in
2020-06-27 00:37:23,580 sagemaker-containers INFO     In


2020-06-27 00:37:54 Uploading - Uploading generated training modelLayer 4 Evaluation Accuracy:
APP_CONF: 0.881578947368421

REFERRAL: 0.875

CODING_CHALLENGE: 0.8333333333333334

INTERVIEW: 0.8811881188118812

OFFER: 0.9523809523809523

REJECTION: 0.8444444444444444

PRE_APP: 0.6818181818181818

POST_OFFER: 0.8461538461538461

2020-06-27 00:37:53,740 sagemaker-containers INFO     Reporting training SUCCESS

2020-06-27 00:38:01 Completed - Training job completed
Training seconds: 74
Billable seconds: 74


In [ ]:
# Deploy relevance model
l3_predictor = layer3.deploy(instance_type='ml.t2.medium', initial_instance_count=1,endpoint_name="pursu-layer3")

# Deploy categorization model
l4_predictor = layer4.deploy(instance_type='ml.t2.medium', initial_instance_count=1,endpoint_name="pursu-layer4")

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-----------------!

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-------------!

In [9]:
"""TEST CASE"""
import boto3
import pandas as pd

# Format the deploy_test data features
request_body = "hi  anon   thank you for applying to asana we appreciate your interest in our mission and for considering us as you take the next steps in your career although the team was impressed with your experience we decided not to move forward with your candidacy at this time  please dont hesitate to stay in touch and reach out in the future if we have another role you feel is a good fit all the best the asana recruiting team"

# create sagemaker client using boto3
client = boto3.client('sagemaker-runtime')

# Specify endpoint and content_type
endpoint_name = "pursu-layer4"
content_type = "text/csv"

# Make call to endpoint
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=request_body
    )

t = response['Body']
j = t.read()
print(j)

b'6'


In [10]:
# Delete created endpoints
l3_predictor.delete_endpoint()
l4_predictor.delete_endpoint()